In [ ]:
# !conda update conda -y
# !conda update conda -y
# !conda install psycopg2 -y

In [ ]:
import psycopg2
from psycopg2 import errorcodes, errors
import os, sys, getpass

In [ ]:
getpass.getuser()

In [ ]:
conn = psycopg2.connect(database="jupyterdb", user=getpass.getuser())


In [ ]:
cur = conn.cursor()
cur.execute("SELECT 1,2,3;")
conn.commit()
cur.fetchall()

Note: If you don't want to have **conn.commit()** everywhere in your code, set

conn.autocommit = True

In [ ]:
# let's cleanup
cur.close(), conn.close()

In [ ]:
# what if we get the database name wrong?
conn = psycopg2.connect(database="doesnotexist", user=getpass.getuser())

In [ ]:
## let's catch the Operational Error for db does not exist now

In [ ]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [ ]:
conn = create_server_connection(database="doesnotexist", user=getpass.getuser())

In [ ]:
#let's make sure the create_server_connection can still connect normally

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = conn.cursor()
cur.execute("SELECT 1,2,3,4,5;")
conn.commit()
cur.fetchall()

In [ ]:
# let's cleanup

cur.close(), conn.close()

In [ ]:
# Let's check error handling on SQL statements

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())

#the context manager will close the cursor when it exists. 
# GOTCHA: YOU MUST CLOSE THE CONNECTION EXPLICITLY (context manager does not close the conn)

with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM NOTABLE;")
        conn.commit()
    except Exception as e:
        print(e)

conn.close()


In [ ]:
# let's write a create/drop database function 

In [ ]:
def create_or_drop_database(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    with conn.cursor() as cur:
        try:
            cur.execute(query)
            print("Database created/dropped successfully.")
        except Exception as e:
            print(e)
        finally:
            #be a good citizen and put stuff back the way you found it
            conn.autocommit = autocommit 

In [ ]:
q = "CREATE DATABASE school4;"

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
create_or_drop_database(conn, q)
conn.close()

In [ ]:
q = "DROP DATABASE school4;"

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
create_or_drop_database(conn, q)
conn.close()

In [ ]:
# let's write an execute query function
# we want our function to return a cursor with the results

def execute_query(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        cur.execute(query)
        print("Query ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = autocommit 
    return cur


In [ ]:
q = "SELECT 1,2,3,4,5"
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = execute_query(conn, q)
print(cur.fetchall())
cur.close()
conn.close()

In [ ]:
q = "SELECT 1,2,3,4,5 FROM doesnotexist"
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = execute_query(conn, q)
cur.close()
conn.close()